In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split

In [0]:


# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    #print(n_col)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная

def sum_neuron(x=None, w=None):
    return np.dot(w.T,x.T )

def sigmoid_complex_neuron(x = None, w = None, bias=0, lymbd = 1):
    #print('X: ',x)
    #print('W: ',w)
    #print('bias: ',bias)
    s = sum_neuron(x=x, w=w).T
    #print(s.shape)
    #print('суммарный уровень входов нейрона на их веса: ',s)
    y = 1 / (1 + np.exp(-s/lymbd))
    return y

def sigmoid_deriv(g):
    return g*(1 - g)

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)


In [0]:
# Подготовка тренировочных данных
# 
iris_ = load_iris()

#данные для входной части модели
iris_data=iris_.data
# данные для выходной части модели
iris_target=iris_.target


In [14]:
iris_.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [15]:
feature=iris_.feature_names
feature

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [16]:
# формирование входных данных
columns = feature
x = pd.DataFrame(data=iris_data, columns=columns)

#сразу сделаем нормализацию

x = normalize(x.as_matrix())

# формирование выходных данных(результатов)
columns = ['Сорт']
y = pd.DataFrame(iris_target, columns=columns)
y = y.as_matrix()
y = y.flatten()
# сразу делаем перевод в векторную форму выходного свойства
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

# покажем размер данных для обучениия
print(X_train.shape)


(100, 4)


In [0]:
def net_MLP(epoch = 3000, neuron = 5 ,lyambd = 0.1, X_train = None, X_test = None, y_train = None, y_test = None, batch_size = 10, n = 0.01):
    # скорость обучения (learning rate)
 
    
    # Инициализация параметров
    w0 = 2*np.random.random((4, neuron)) - 1 # для входного слоя   - 4 входа, neuron выхода
    w1 = 2*np.random.random((neuron, 3)) - 1 # для внутреннего слоя - neuron входов, 3 выхода

    # массив для ошибок
    errors = []
    errors_test=[]
    df=[]

    
    
    # массив для ошибок, чтобы потом построить график
    layer0 = X_train
    layer1 = sigmoid_complex_neuron(x= layer0, w=w0)
    layer2 = sigmoid_complex_neuron(x=layer1, w=w1)
    layer2_error = (y_train - layer2)

    # Сделаем оценку
    Q = np.mean(np.abs(layer2_error)).copy()
    
    # процесс обучения
    for i in range(epoch):
        #print(i)
        # ПРЯМОЙ ПРОХОД
    
        # создадим список примеров для обучения
        j = np.random.randint(0,X_train.shape[0],(batch_size))
        layer0 = X_train[j,:]
        layer1 = sigmoid_complex_neuron(x= layer0, w=w0)
        layer2 = sigmoid_complex_neuron(x=layer1, w=w1)
        #print(layer2.shape, layer1.shape)
        
        # Ошибка на выходе
        layer2_error = y_train[j] - layer2
        #print(layer2_error.shape)
        
        
        Q=Q*(1-lyambd)+lyambd*np.mean(np.abs(layer2_error))
        errors.append(Q.copy())
    
        # Производная
        dlayer2 = sigmoid_deriv(layer2)
        dlayer1 = sigmoid_deriv(layer1)
        #print('производнаяя для нейронов выходного слоя: ',dlayer2.shape)
        #print('производнаяя для нейронов скрытого слоя: ', dlayer1)
        
        # поправки дляя обратного прохода    
        layer2_delta = layer2_error*dlayer2 
    
    
        layer1_error = np.dot(layer2_delta , w1.T)
                
        layer1_delta = layer1_error*dlayer1 
        
        #  вычисляем поправку для параметров нейронов            
        dw1 = np.dot(layer2_delta.reshape((layer2_delta.shape[1], layer2_delta.shape[0])),layer1) * n
        dw0 = np.dot(layer1_delta.reshape((neuron, layer1_delta.shape[0])),layer0) * n
       
        w1 += dw1.T
        w0 += dw0.T
        
        error = np.mean(np.abs(layer2_error))
        accuracy = (1 - error.copy()) * 100
    
        # оценка сети на тестах
        layer0_test = X_test
        layer1_test = sigmoid_complex_neuron(x= layer0_test, w=w0)
        layer2_test = sigmoid_complex_neuron(x=layer1_test, w=w1)
        
        
        layer2_error_test = y_test - layer2_test
    
        error = np.mean(np.abs(layer2_error_test))
        errors_test.append(error.copy())
    
        accuracy_test = (1 - error.copy()) * 100
        
    return accuracy, accuracy_test, w0, w1, errors_test, errors
    

In [0]:
accuracy, accuracy_test, w0, w1, errors_test, errors = net_MLP(neuron = 5 ,lyambd = 0.02, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test, batch_size = 1, n = 0.1)

In [42]:
def get_mean_accuracy(batch_s = 1, neuron = 5 ,lyambd = 0.02):
    accuracies = []            
    accuracies_test = []
 
    for i in range(10):
        accuracy, accuracy_test, w0, w1, errors_test, errors = net_MLP(neuron=neuron,lyambd=lyambd, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test, batch_size=batch_s, n = 0.1)  
        accuracies.append(accuracy.copy())
        accuracies_test.append(accuracy_test.copy()) 

    accuracies = np.array(accuracies)           
    accuracies_test = np.array(accuracies_test)

    accuracy = accuracies.mean()            
    accuracy_test = accuracies_test.mean()
    std_accuracy = accuracies.std()            
    std_accuracy_test = accuracies_test.std()
    return accuracy, accuracy_test, std_accuracy, std_accuracy_test

accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(batch_s = 1)
print("batch_size = 1")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

batch_size = 1
Средняя точность нейронной сети 59.41%
Средняя точность нейронной сети test 60.64%
STD нейронной сети 4.8%
STD нейронной сети test 3.11%


In [43]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(batch_s = 10)
print("batch_size = 10")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

batch_size = 10
Средняя точность нейронной сети 57.99%
Средняя точность нейронной сети test 58.43%
STD нейронной сети 2.07%
STD нейронной сети test 2.24%


In [44]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(batch_s = 20)

print("batch_size = 20")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

batch_size = 20
Средняя точность нейронной сети 55.15%
Средняя точность нейронной сети test 55.22%
STD нейронной сети 0.37%
STD нейронной сети test 0.49%


In [45]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(batch_s = 50)

print("batch_size = 50")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

batch_size = 50
Средняя точность нейронной сети 55.35%
Средняя точность нейронной сети test 55.64%
STD нейронной сети 0.46%
STD нейронной сети test 0.3%


Вывод: точность НС в целом мало зависит от batch_size. Четкой закономерности не прослеживается.

In [47]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(lyambd = 0.01)

print("lyambd = 0.01")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

lyambd = 0.01
Средняя точность нейронной сети 64.52%
Средняя точность нейронной сети test 61.68%
STD нейронной сети 4.45%
STD нейронной сети test 1.5%


In [48]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(lyambd = 0.1)

print("get_mean_accuracy(lyambd = 0.1)")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

get_mean_accuracy(lyambd = 0.1)
Средняя точность нейронной сети 59.23%
Средняя точность нейронной сети test 61.38%
STD нейронной сети 2.75%
STD нейронной сети test 2.7%


In [49]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(lyambd = 0.5)

print("lyambd = 0.5")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

lyambd = 0.5
Средняя точность нейронной сети 61.73%
Средняя точность нейронной сети test 61.53%
STD нейронной сети 5.93%
STD нейронной сети test 2.37%


In [50]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(lyambd = 1)

print("lyambd = 1")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

lyambd = 1
Средняя точность нейронной сети 59.92%
Средняя точность нейронной сети test 60.32%
STD нейронной сети 4.6%
STD нейронной сети test 3.48%


Вывод: точность НС в целом не зависит от lyambd

In [51]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(neuron = 1)

print("neuron = 1")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

neuron = 1
Средняя точность нейронной сети 55.44%
Средняя точность нейронной сети test 55.26%
STD нейронной сети 3.04%
STD нейронной сети test 0.17%


In [53]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(neuron = 5)

print("neuron = 5")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

neuron = 5
Средняя точность нейронной сети 60.76%
Средняя точность нейронной сети test 61.26%
STD нейронной сети 4.05%
STD нейронной сети test 1.68%


In [54]:
accuracy, accuracy_test, std_accuracy, std_accuracy_test = get_mean_accuracy(neuron = 50)

print("neuron = 50")
print("Средняя точность нейронной сети " + str(round(accuracy,2)) + "%")
print("Средняя точность нейронной сети test " + str(round(accuracy_test,2)) + "%")
print("STD нейронной сети " + str(round(std_accuracy,2)) + "%")
print("STD нейронной сети test " + str(round(std_accuracy_test,2)) + "%")

neuron = 50
Средняя точность нейронной сети 63.89%
Средняя точность нейронной сети test 72.01%
STD нейронной сети 10.11%
STD нейронной сети test 1.42%


Вывод: Нс не переобучается и стабильно увеличивает точность с увеличением кол-ва нейронов

Простая нейросеть на Keras. Решил проверить полученные закономерности с использованием керас. 

In [56]:
# The full neural network code!
###############################
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn import datasets
# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split


# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)


# преобразование массива в необходимый вид
def from_one_hot(binarized):
    Y = []
    for item in binarized:
        for i, binary in enumerate(item):
            if binary == 1:
                Y.append(i)
                break
    return Y            


iris_data = datasets.load_iris().data
iris_labels = datasets.load_iris().target

# формирование входных данных
x = normalize(iris_data)

# формирование выходных данных(результатов)
y = to_categorical(iris_labels) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

count_neurons = 5
# Build the model.
model = Sequential([
  Dense(count_neurons, activation='relu', input_shape=(4,)),
  Dense(3, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=1,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при batch = 1 ',results[1])
# Save the model to disk.
#model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
predictions = model.predict(X_test[:20])

# Print our model's predictions.
print(np.argmax(predictions, axis=1))

# Check our predictions against the ground truths.
#print(y_test[:20])
print(from_one_hot(y_test[:20]))



Epoch 1/1000
100/100 [==============================] - 0s 3ms/step - loss: 1.1183 - acc: 0.3800
Epoch 2/1000
100/100 [==============================] - 0s 956us/step - loss: 1.1042 - acc: 0.3800
Epoch 3/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0967 - acc: 0.3800
Epoch 4/1000
100/100 [==============================] - 0s 998us/step - loss: 1.0910 - acc: 0.3800
Epoch 5/1000
100/100 [==============================] - 0s 938us/step - loss: 1.0878 - acc: 0.3800
Epoch 6/1000
100/100 [==============================] - 0s 994us/step - loss: 1.0833 - acc: 0.3800
Epoch 7/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0804 - acc: 0.3800
Epoch 8/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0765 - acc: 0.3800
Epoch 9/1000
100/100 [==============================] - 0s 954us/step - loss: 1.0740 - acc: 0.3800
Epoch 10/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0714 - acc: 0.3800
Epoch 11/1000
100/1

In [58]:
# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=10,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при batch = 10: ',results[1])


Epoch 1/1000
100/100 [==============================] - 0s 225us/step - loss: 0.0991 - acc: 0.9500
Epoch 2/1000
100/100 [==============================] - 0s 156us/step - loss: 0.0989 - acc: 0.9500
Epoch 3/1000
100/100 [==============================] - 0s 169us/step - loss: 0.0989 - acc: 0.9500
Epoch 4/1000
100/100 [==============================] - 0s 151us/step - loss: 0.0988 - acc: 0.9500
Epoch 5/1000
100/100 [==============================] - 0s 163us/step - loss: 0.0989 - acc: 0.9500
Epoch 6/1000
100/100 [==============================] - 0s 160us/step - loss: 0.0988 - acc: 0.9500
Epoch 7/1000
100/100 [==============================] - 0s 168us/step - loss: 0.0988 - acc: 0.9500
Epoch 8/1000
100/100 [==============================] - 0s 141us/step - loss: 0.0992 - acc: 0.9500
Epoch 9/1000
100/100 [==============================] - 0s 130us/step - loss: 0.0987 - acc: 0.9500
Epoch 10/1000
100/100 [==============================] - 0s 158us/step - loss: 0.0988 - acc: 0.9500
Epoch 11/

In [59]:
# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=20,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при batch = 20: ',results[1])


Epoch 1/1000
100/100 [==============================] - 0s 181us/step - loss: 0.0922 - acc: 0.9500
Epoch 2/1000
100/100 [==============================] - 0s 134us/step - loss: 0.0921 - acc: 0.9500
Epoch 3/1000
100/100 [==============================] - 0s 112us/step - loss: 0.0922 - acc: 0.9500
Epoch 4/1000
100/100 [==============================] - 0s 105us/step - loss: 0.0922 - acc: 0.9500
Epoch 5/1000
100/100 [==============================] - 0s 106us/step - loss: 0.0923 - acc: 0.9500
Epoch 6/1000
100/100 [==============================] - 0s 99us/step - loss: 0.0922 - acc: 0.9500
Epoch 7/1000
100/100 [==============================] - 0s 101us/step - loss: 0.0923 - acc: 0.9500
Epoch 8/1000
100/100 [==============================] - 0s 109us/step - loss: 0.0922 - acc: 0.9500
Epoch 9/1000
100/100 [==============================] - 0s 118us/step - loss: 0.0924 - acc: 0.9500
Epoch 10/1000
100/100 [==============================] - 0s 106us/step - loss: 0.0922 - acc: 0.9500
Epoch 11/1

In [60]:
# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=50,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при batch = 50: ',results[1])


Epoch 1/1000
100/100 [==============================] - 0s 116us/step - loss: 0.0894 - acc: 0.9500
Epoch 2/1000
100/100 [==============================] - 0s 73us/step - loss: 0.0894 - acc: 0.9500
Epoch 3/1000
100/100 [==============================] - 0s 78us/step - loss: 0.0894 - acc: 0.9500
Epoch 4/1000
100/100 [==============================] - 0s 98us/step - loss: 0.0894 - acc: 0.9500
Epoch 5/1000
100/100 [==============================] - 0s 64us/step - loss: 0.0894 - acc: 0.9500
Epoch 6/1000
100/100 [==============================] - 0s 53us/step - loss: 0.0894 - acc: 0.9500
Epoch 7/1000
100/100 [==============================] - 0s 61us/step - loss: 0.0894 - acc: 0.9500
Epoch 8/1000
100/100 [==============================] - 0s 58us/step - loss: 0.0894 - acc: 0.9500
Epoch 9/1000
100/100 [==============================] - 0s 59us/step - loss: 0.0894 - acc: 0.9500
Epoch 10/1000
100/100 [==============================] - 0s 76us/step - loss: 0.0894 - acc: 0.9500
Epoch 11/1000
100/

Вывод: точность НС не зависит от batch_size

In [62]:
count_neurons = 1
# Build the model.
model = Sequential([
  Dense(count_neurons, activation='relu', input_shape=(4,)),
  Dense(3, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=1,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при количестве нейронов = 1: ',results[1])


Epoch 1/1000
100/100 [==============================] - 0s 4ms/step - loss: 1.0987 - acc: 0.3200
Epoch 2/1000
100/100 [==============================] - 0s 984us/step - loss: 1.0978 - acc: 0.3800
Epoch 3/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0971 - acc: 0.3800
Epoch 4/1000
100/100 [==============================] - 0s 975us/step - loss: 1.0963 - acc: 0.3800
Epoch 5/1000
100/100 [==============================] - 0s 953us/step - loss: 1.0956 - acc: 0.3800
Epoch 6/1000
100/100 [==============================] - 0s 929us/step - loss: 1.0951 - acc: 0.3800
Epoch 7/1000
100/100 [==============================] - 0s 935us/step - loss: 1.0947 - acc: 0.3800
Epoch 8/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0941 - acc: 0.3800
Epoch 9/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0940 - acc: 0.3800
Epoch 10/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0936 - acc: 0.3800
Epoch 11/1000
100/1

In [63]:
count_neurons = 5
# Build the model.
model = Sequential([
  Dense(count_neurons, activation='relu', input_shape=(4,)),
  Dense(3, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=1,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при количестве нейронов = 5: ',results[1])


Epoch 1/1000
100/100 [==============================] - 0s 4ms/step - loss: 1.1400 - acc: 0.3400
Epoch 2/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.1188 - acc: 0.3400
Epoch 3/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.1063 - acc: 0.3400
Epoch 4/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0980 - acc: 0.3400
Epoch 5/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0935 - acc: 0.3400
Epoch 6/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0884 - acc: 0.3400
Epoch 7/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0838 - acc: 0.3400
Epoch 8/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0776 - acc: 0.3400
Epoch 9/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0624 - acc: 0.3400
Epoch 10/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0498 - acc: 0.3400
Epoch 11/1000
100/100 [======

In [64]:
count_neurons = 50
# Build the model.
model = Sequential([
  Dense(count_neurons, activation='relu', input_shape=(4,)),
  Dense(3, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=1,
)

# Evaluate the model.
results = model.evaluate(
  X_test,
  y_test
)
print('точность при количестве нейронов = 1: ',results[1])


Epoch 1/1000
100/100 [==============================] - 1s 5ms/step - loss: 1.0737 - acc: 0.3800
Epoch 2/1000
100/100 [==============================] - 0s 1ms/step - loss: 1.0023 - acc: 0.4400
Epoch 3/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.9371 - acc: 0.5400
Epoch 4/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.8728 - acc: 0.6600
Epoch 5/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.8028 - acc: 0.7500
Epoch 6/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.7406 - acc: 0.8800
Epoch 7/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.6829 - acc: 0.7100
Epoch 8/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.6365 - acc: 0.8100
Epoch 9/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.5967 - acc: 0.8000
Epoch 10/1000
100/100 [==============================] - 0s 1ms/step - loss: 0.5646 - acc: 0.9300
Epoch 11/1000
100/100 [======

Вывод: точность увеличивается при увеличении количества нейронов. При 5 нейронах достигается насыщение и больше точность сети не растет.